# Assignment 1

Lucas Sylvester - s214636 <br>
Lucas Emcken - s214625 <br>
Lila <br>
**Github** \
https://github.com/LucasEmcken/SocialScienceAssignments

**Contribution statement** \
Everybody made all exercises before hand.\
The final assignment was made by combining and \
discussing different approaches to the each exercise.\
\
The git history is not representative as it was made using\
 VS code Live Share to work in unison on one computer in real time.





## Part 1 - Web-scraping


In [39]:
import bs4
import requests

from fuzzywuzzy import process

In [40]:
url = "https://ic2s2-2023.org/program"

response = requests.get(url)
soup = bs4.BeautifulSoup(response.content, 'html.parser')

#get content in section with id "main"
main = soup.find_all('section', id='main')

#get the list of each <\i> tag in the main section
authors_list = []

for section in main:
    #find italic tags since authors are in italic
    items = section.find_all('i')
    for item in items:
        authors = item.text
        for author in authors.split(","):
            author_strip = author.strip()
            #remove Chair: in the beginning of the name if it exists
            if author_strip.startswith("Chair:"):
                author_strip = author_strip[6:]
            authors_list.append(author_strip)

#remove duplicates
authors_list = list(set(authors_list))
print(f"initial length of authors list: {len(authors_list)}")

initial length of authors list: 1524


In [41]:
#use fuzzywuzzy to remove similar authors to further prevent misspelling and duplicates
def remove_similar_authors(authors, similarity_threshold = 85):
    unique_authors = []
    for author in authors:
        # Check if the author is too similar to any already in unique_authors
        if not unique_authors:
            unique_authors.append(author)
        else:
            similarities = [process.extractOne(author, [ua])[1] for ua in unique_authors]
            if all(sim < similarity_threshold for sim in similarities):
                unique_authors.append(author)
    return unique_authors

# #sort authors list
# authors_list.sort()

# #split authors list into lists of n authors to speed up the process
# n = 100
# split_authors = [authors_list[i:i + n] for i in range(0, len(authors_list), n)]
# cleaned_authors = []
# for split in split_authors:
#     cleaned_authors += remove_similar_authors(split)

# print(f"length of cleaned authors: {len(cleaned_authors)}")
# # Save the list to a file with UTF-8 encoding to handle special characters
# with open("authors.txt", "w", encoding="utf-8") as f:
#     for author in cleaned_authors:
#         f.write(author + "\n")

**1. How many researchers did you get?**\
Initially, we retrieved 1524 authors.
By sortin the list and cleaning using a Levenshtein distance similarity measure we got 1394 authors.
The threshhold for if names where the same where put at 85, all the cases we observed with this threshhold semeed to be correct. It might have missed a few really significand misspellings, but as in some of the examples below, it will have cleaned the majority of misspellings and variations.

**2. Explain the process you followed to web-scrape the page** \
For scraping, we noticed authors names were italicized so we retrieved this text. This also contained people with the Chair: tag in front, so we removed tag this by a simple comparison while retaining the name in the set. After this we used fuzzywuzzy to extract and compare similar author names. This removes duplicates as a result of misspellings or alternative names and such.

**Example of similar names removed:** \
Alessandro Flamini \
Alessandro Flammini \
similarity:97 \
\
Alexander Gates \
Alexander J Gates \
similarity:94 \
\
Ana Maria Jaramillo \
Ana María Jaramillo \
similarity:95\
\
Anne C Kroon\
Anne C. Kroon\
similarity: 96\
\
Anne C Kroon\
Anne Kroon\
similarity:91

## Part 2 - Ready Made vs Custom Made Data
**1. What are the pros and cons** \
...
**2. How can these differences influence interpretation (max 150. words)**\
...

## Part 3 - Gathering Research Articles using the OpenAlex API

In [42]:
import pandas as pd
import tqdm
import concurrent.futures
import ast

In [43]:
#load author txt
def load_author_txt(file_path):
    with open(file_path, 'r', encoding="utf-8") as f:
        lines = f.readlines()
        authors = []
        for line in lines:
            line = line.strip()
            if len(line) > 0:
                authors.append(line)
    return authors

authors = load_author_txt('authors.txt')

In [44]:
endpoint = "https://api.openalex.org/authors?search="


def get_author_response(author):
    url = endpoint + author
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

#helper function to get the concepts from the author dictionary and the intercection if both required concept groups are present
def get_concepts(author_dict):
    concepts = [[c.get('display_name') for c in author_dict.get('results')[a].get('x_concepts')] for a in range(len(author_dict.get('results')))]
    return concepts

def get_best_match(concepts):
    target_1 = ['Sociology', 'Psychology', 'Economics', 'Political Science']
    target_2 = ['Mathematics', 'Physics', 'Computer Science']

    for i, author in enumerate(concepts):
        #get authors with concepts in target_1 AND target_2
        if (x in author for x in target_1) and (x in author for x in target_2):
            return i
    
    return 0

def get_info(auth_info):
    concepts = get_concepts(auth_info)
    c_index = get_best_match(concepts)
    
    #get their id
    author_id = auth_info.get('results')[c_index].get('id')[-11:]
    #get their display_name
    display_name = auth_info.get('results')[c_index].get('display_name')
    #get their works_api_url
    works_api_url = "https://api.openalex.org/works?filter=author.id:" + author_id
    #get their h_index
    h_index = auth_info.get('results')[c_index].get('summary_stats').get('h_index')
    #get their works count
    works_count = auth_info.get('results')[c_index].get('works_count')
    #get their cited_by_count
    cited_by_count = auth_info.get('results')[c_index].get('cited_by_count')
    
    #get their country code
    country_code = auth_info.get('results')[c_index].get('last_known_institution').get('country_code')

    return {
        'author_id': author_id,
        'display_name': display_name,
        'works_api_url': works_api_url,
        'h_index': h_index,
        'works_count': works_count,
        'cited_by_count': cited_by_count,
        'country_code': country_code
    }

def get_author_df(authors,*, n_workers = 3, load_if_exists = True, save_path = 'authors_df.csv'):
    
    if load_if_exists:
        try:
            authors_df = pd.read_csv(save_path)
            print(f'Loaded {len(authors_df)} authors from {save_path}')
            return authors_df
        except:
            print(f'Failed to load {save_path}, will create a new one..')
            
    authors_df = pd.DataFrame(columns=['author_id', 'display_name', 'works_api_url', 'h_index', 'works_count', 'cited_by_count', 'country_code'])
    err_authors = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=n_workers) as executor:
        future_to_author = {executor.submit(get_author_response, author): author for author in authors}
        futures = tqdm.tqdm(concurrent.futures.as_completed(future_to_author), total=len(authors))
        for future in futures:
            author = future_to_author[future]
            try:
                res = future.result()
                if res:
                    try:
                        info = get_info(res)
                        authors_df = pd.concat([authors_df, pd.DataFrame([info])])
                    except:
                        err_authors.append(author)
                else:
                    err_authors.append(author)
            except Exception as e:
                err_authors.append(author)
    print(f'Done!, Failed authors: {len(err_authors)}')
    #save to csv

    authors_df.to_csv(save_path, index=False)
    return authors_df

authors_df = get_author_df(authors)

Loaded 1237 authors from authors_df.csv


In [45]:
pruned_authors = authors_df[(authors_df['works_count'] > 5) & (authors_df['works_count'] < 5000) &
                              (authors_df['cited_by_count'] > 10)]

In [46]:
def get_intercept(concepts):
    target = ['Sociology', 'Psychology', 'Economics', 'Political Science']
    
    if any(x in concepts for x in target):
        return True
    else:
        return False
    
def get_works(url):
    
    # response = requests.get(url)
    filters = 'concepts_count:%3E1,authors_count:%3C10,concept.id:C33923547|C121332964|c41008148&per-page=200'
    filter_url = url + filters
    # return response.json()
    response = requests.get(filter_url, timeout=10).json()
    return_dict = {
        'id': [],
        'publication_year': [],
        'cited_by_count': [],
        'author_ids': [],
        'title': [],
        'abstract_inverted_index': []
    }
    
    page = 1
    # print(len(response.get('results')))
    while len(response.get('results')) != 0:
        for paper in response.get('results'):
            concepts = [c['display_name'] for c in paper['concepts']]
            if get_intercept(concepts):
                # return_dict['id'] = paper['id']
                return_dict['id'].append(paper['id'])
                return_dict['publication_year'].append(paper['publication_year'])
                return_dict['cited_by_count'].append(paper['cited_by_count'])
                
                #authorships
                return_dict['author_ids'].append([id.get('author').get('id') for id in paper['authorships']])
                
                
                return_dict['title'].append(paper['title'])
                # return_dict['abstract_inverted_index'] = ' '.join(list(paper['abstract_inverted_index'].keys())) if paper['abstract_inverted_index'] else []
                return_dict['abstract_inverted_index'].append(paper['abstract_inverted_index'])
        
        page += 1
        
        filters = f'concepts_count:%3E1,authors_count:%3C10,concept.id:C33923547|C121332964|c41008148&per-page=200&page={page}'
        filter_url = url + filters
        response = requests.get(filter_url).json()
                
    return return_dict
    

def combine_works(df, n_workers=3, load_if_exists=True, save_path='combined_works.csv'):
    if load_if_exists:
        try:
            combined_df = pd.read_csv(save_path)
            print('works found, formatting..')
            combined_df = combined_df.dropna()
            combined_df['author_ids'] = combined_df['author_ids'].apply(lambda x: ast.literal_eval(x))
            combined_df['abstract_inverted_index'] = combined_df['abstract_inverted_index'].apply(ast.literal_eval)
            print(f'Loaded {len(combined_df)} works from {save_path}')
            return combined_df
        except:
            print(f'Failed to load {save_path}, will create a new one..')
    
    combined_df = pd.DataFrame()
    
    failed = []
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=n_workers) as executor:
        results = list(tqdm.tqdm(executor.map(get_works, df['works_api_url']), total=len(df)))
        for i in range(len(results)):
            try:
                works_df = pd.DataFrame(results[i])
                combined_df = pd.concat([combined_df, works_df])
            except:
                failed.append(i)
    
    print(f'Done!, Failed authors: {len(failed)}')
    #save to csv
    combined_df.to_csv(save_path, index=False)
    
    return combined_df

full_df = combine_works(pruned_authors, n_workers=5, load_if_exists=True)

works found, formatting..
Loaded 30287 works from combined_works.csv


In [47]:
full_df = full_df[full_df['cited_by_count'] > 10]
full_df = full_df[full_df['author_ids'].apply(lambda x: len(x) < 10)]

#split into IC2S2 papers and IC2S2 abstracts
ic2s2_papers = full_df[['id', 'publication_year', 'cited_by_count', 'author_ids']]
ic2s2_abstracts = full_df[['id', 'title', 'abstract_inverted_index']]

print(len(ic2s2_papers))

9394


In [48]:
#get the authors of the papers
def get_paper_authors(paper):
    author_ids = []
    for ids in paper['author_ids']:
        author_ids += ids
    author_ids = list(set(author_ids))
    
    api_authors = []
    for id in author_ids:
        api_authors.append(f'https://api.openalex.org/authors/{id[-11:]}')
    
    return api_authors

co_authors_apis = get_paper_authors(ic2s2_papers)

def get_author_info(url):
    
    try:
        response = requests.get(url, timeout=10).json()
        return_dict = {
            'id': "https://openalex.org/" + url[-11:],
            'display_name': response.get('display_name'),
            'works_api_url': response.get('works_api_url'),
            'h_index': response.get('summary_stats').get('h_index'),
            'works_count': response.get('works_count'),
            'country_code': response.get('last_known_institution').get('country_code')
        }
        return return_dict
    except:
        return_dict = {
            'display_name': None,
            'works_api_url': None,
            'h_index': None,
            'works_count': None,
            'country_code': None
        }
    
        return return_dict

def combine_authors(author_urls, n_workers=3, load_if_exists=True, save_path='co_authors.csv'):
    if load_if_exists:
        try:
            combined_df = pd.read_csv(save_path)
            combined_df = combined_df.dropna()
            print(f'Loaded {len(combined_df)} authors from {save_path}')
            return combined_df
        except:
            print(f'Failed to load {save_path}, will create a new one..')
    combined_df = pd.DataFrame()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        results = list(tqdm.tqdm(executor.map(get_author_info, author_urls), total=len(author_urls)))
        for i in range(len(results)):
            author_df = pd.DataFrame(results[i], index=[0])
            combined_df = pd.concat([combined_df, author_df])
    #save to csv
    combined_df = combined_df.dropna()
    combined_df.to_csv(save_path, index=False)
    
    return combined_df

co_authors_df = combine_authors(co_authors_apis[:1000], load_if_exists=True)

Loaded 989 authors from co_authors.csv


In [49]:
#create an authorship df with the display names from the co_authors_df compared with the author_ids from the ic2s2_papers
authorship_df = pd.DataFrame(columns=['paper_id', 'author_ids', 'display_names', 'countries', 'citation_count', 'publication_year'])

for i in range(len(ic2s2_papers)):
    paper = ic2s2_papers.iloc[i]
    author_ids = paper['author_ids']
    display_names = []
    countries = []
    for id in author_ids:
        author = co_authors_df[co_authors_df['id'] == id]
        if len(author) > 0:
            display_names.append(author['display_name'].values[0])
            countries.append(author['country_code'].values[0])
    authorship_df = pd.concat([authorship_df, pd.DataFrame({'paper_id': paper['id'], 'author_ids': [author_ids], 'display_names': [display_names], 'countries': [countries], 'citation_count': [paper['cited_by_count']], 'publication_year': [paper['publication_year']]})])

C:\Users\lucas\AppData\Local\Temp\ipykernel_13700\127905344.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  authorship_df = pd.concat([authorship_df, pd.DataFrame({'paper_id': paper['id'], 'author_ids': [author_ids], 'display_names': [display_names], 'countries': [countries], 'citation_count': [paper['cited_by_count']], 'publication_year': [paper['publication_year']]})])


In [50]:
#sort by length of display names
authorship_df['len_display_names'] = authorship_df['display_names'].apply(lambda x: len(x))
authorship_df = authorship_df.sort_values(by='len_display_names', ascending=False)

**Dataset summary.** \
**How many works are listed in your IC2S2 papers dataframe?**\
**How many unique researchers have co-authored these works?**\
**Efficiency in code.**\
**Describe the strategies you implemented to make your code more efficient.** \
We implementet the following strategies:
1. Filtering directly in query to return less more relevant results, instead of filtering in code.
2. Multithreading
3. 
**How did your approach affect your code's execution time? (answer in max 150 words)**\
Significantly?
**Filtering Criteria and Dataset Relevance** \
**Reflect on the rationale behind setting specific thresholds for the total number of works by an author, the citation count, the number of authors per work, and the relevance of works to specific fields. How do these filtering criteria contribute to the relevance of the dataset you compiled? Do you believe any aspects of Computational Social Science research might be underrepresented or overrepresented as a result of these choices? (answer in max 150 words)**\

The threshholding can be used to filter off mistakes. The api is not perfect.\ and ther instances of duplicates of authors, or authors who have several thousand works, aliases etc. Its for instance reasonable to that authors with more than 5000 works, might have some mistakes in their record or similar.
\
The limit of 10 authors per work, can be used to at least keep
 the extension of the author list relevant. If there are more than 10 authors for a work the majority of them might have had very little part in work and are therefore probaly not relevant. The authors that are relevant, would likely show up on other works and be included through there. Likewise with the minimum citation of 10, it resonable to say that established relevant work have at least 10 citations.

However, brand new reasearch that just been published might be under represented. It might be that a new paper has not yet been reviewed and cited by other papers. Likewise older papers, that are less relevant, might eventually get 10 citations even though its less relevant than newer papers.

\



**Sociology OR Psychology OR Economics OR Political Science AND Mathematics OR Physics OR Computer Science**







## Part 4 - The Network of Computational Social Scientists

In [51]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
from collections import Counter
import json

In [52]:
#create a weighted edgelist weighted edgelist where each list element is a tuple containing three elements: the author ids of two collaborating authors and the total number of papers they've co-authored. Ensure each author pair is listed only once.

weighted_edgelist = []

def weighted_creation (df): 

    #list of all author    
    auth_pairs = []
    
    #get the list of authors in the dataframe 

    for _, row in df.iterrows():
        authors = row['author_ids']
    
        for pair in combinations(authors, 2): 
            
            auth_pairs.append(tuple(sorted(pair))) 

    #Count the occurrences of each author pair to determine the number of co-authored papers
    
    edge_weights = Counter(auth_pairs)

    # Convert the edge weights to a list of tuples in the format (author_id_1, author_id_2, weight)
    weighted_edgelist = [(pair[0], pair[1], weight) for pair, weight in edge_weights.items()]

    return weighted_edgelist

weighted_creation(authorship_df)

def graph_construction (weighted_edgelist):
    # Create a new graph
    G = nx.Graph()
    
    # Add edges to the graph
    G.add_weighted_edges_from(weighted_edgelist)
    return G

def node_attributes(G, df, file_path): 
    
    for _, row in df.iterrows():
        
        # Check if the author is a node in the graph
        if G.has_node(row['author_ids']):
            # Set the author's attributes
            nx.set_node_attributes(G, {row['author_id']: {
                'display_name': row['display_name'],
                'country': row['country'],
                'citation_count': row['citation_count'],
                'year_first_publication': row['year_first_publication']
            }})

    graph_data = nx.node_link_data(G)
    
    # Write the graph data to a JSON file
    with open(file_path, 'w') as f:
        json.dump(graph_data, f)
    print(f"Network graph saved as {file_path}")

In [54]:
weighted_edgelist = weighted_creation(authorship_df)
graph = graph_construction(weighted_edgelist)
node_attributes(graph, authorship_df, 'network_graph.json')

# #plot the graph
# def plot_graph(G, file_path):
#     plt.figure(figsize=(20, 20))
#     pos = nx.spring_layout(G, k=0.05)
#     nx.draw(G, pos, node_size=10, width=0.1)
#     plt.savefig(file_path)
#     print(f"Network graph saved as {file_path}")

# plot_graph(graph, 'network_graph.png')

Network graph saved as network_graph.json


**Preliminary Network Analysis**

In [55]:
import numpy as np

In [56]:
print("1. Network metrics")
print(f"Total number( of nodes: {len(graph.nodes)}, total number of links: {len(graph.edges)}")
density = nx.density(graph)
print(f"Network density: {density}, The network is sparse, since only {np.round((density)*100, 2)}% of all possible links are present.")
print(f"The network is connected: {nx.is_connected(graph)}")
print(f"The network has {nx.number_connected_components(graph)} connected components.")
print(f"The size of the largest connected component: {len(max(nx.connected_components(graph), key=len))}")
print(f"The networks has {len(list(nx.isolates(graph)))} isolated nodes.")


1. Network metrics
Total number( of nodes: 12676, total number of links: 39559
Network density: 0.0004924306786092839, The network is sparse, since only 0.05% of all possible links are present.
The network is connected: False
The network has 235 connected components.
The size of the largest connected component: 9818
The networks has 0 isolated nodes.


In [57]:
print(f"2. Degree analysis")
avg_degree = np.mean(list(dict(graph.degree()).values()))
avg_degree = np.round(avg_degree, 2)
median_degree = np.median(list(dict(graph.degree()).values()))
minimum_degree = min(dict(graph.degree()).values())
maximum_degree = max(dict(graph.degree()).values())
print(f"Average degree: {avg_degree}, median degree: {median_degree}, minimum degree: {minimum_degree}, maximum degree: {maximum_degree}")

print("3. Top authors")
top_authors = sorted(dict(graph.degree()).items(), key=lambda x: x[1], reverse=True)[:10]
for author, degree in top_authors[:5]:
    print(f"{author}: {degree}")


2. Degree analysis
Average degree: 6.24, median degree: 5.0, minimum degree: 1, maximum degree: 184
3. Top authors
https://openalex.org/A5088141761: 184
https://openalex.org/A5044944954: 155
https://openalex.org/A5075080019: 151
https://openalex.org/A5029100305: 138
https://openalex.org/A5017914184: 135


What area do the authors specialize in? (max 150 words)